In [9]:
import os
from dotenv import load_dotenv
# 올바른 임포트 경로로 수정
from langchain_community.document_loaders import PyMuPDFLoader # PyPDFLoader 대신 사용
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.retrievers.multi_query import MultiQueryRetriever

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("openai api 키가 없습니다. 한번더 확인 부탁드립니다.")

# 환경 변수 설정
os.environ['OPENAI_API_KEY'] = openai_api_key

# Loader: 에러가 자주 나는 PyPDFLoader 대신 PyMuPDFLoader 사용
loader = PyMuPDFLoader("sample.pdf")

# 데이터를 불러오고 분할
# load_and_split()에 splitter를 인자로 줄 수 있습니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
pages = loader.load_and_split(text_splitter=text_splitter)

texts = text_splitter.split_documents(pages)

# 결과 확인
print(f"총 {len(pages)}개의 페이지(청크)가 로드되었습니다.")
if pages:
    print("첫 번째 페이지 내용 일부:", pages[0].page_content[:100])

총 7개의 페이지(청크)가 로드되었습니다.
첫 번째 페이지 내용 일부: 에코프로 공급물량 신청 사이트?…"사
기입니다"
"60세 이상 고용률 높이면 잠재성장률
증가"…노동시장 유연성 전제돼야
'오염물질·온실가스 무배출' 수소차 첫
등록 5년 만에 10


In [12]:
# Embedding
embeddings_model = OpenAIEmbeddings()

#load it into Chroma
vectorstore = Chroma.from_documents(texts,embeddings_model)

chroma_retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 1, 'fetch_k': 4}
)


## BM25
- 전통적 정보검색 모델, TF-IDF 와 유사한 방식을 동작
- 사용자 입력 질의와 문서간 관련성을 점수화하여 문서 순위 매김
- 문서 내 검색단어의 빈도와, 문서 길이를 고려, 과도하게 긴 문서가 높은 문서를 받지 않도록 함.
- 주요 특징
    - 단어 빈도: 특정 단어가 얼마나 자주 등장하는 지 계산
    - 역문서 빈도: 특정 단어가 전체 문서 집합에서 얼마나 흔하지 않은지 측정. 흔하지 않은 단어일수록 (그 문서에만 나타날수록) 중요한 정보로 간주
    - 문서 길이 정규화: 문서 길이 보정. 너무 긴 문서는 불리하게, 짧은 문서는 유리하게 가중치 조절.  문서 길이 길면, 길다는 이유로 단어가 자주 등장하여 관련성과 무관하게 점수 높게 받는 현상 방지

$$score(D, Q) = \sum_{q \in Q} \text{IDF}(q) \cdot \frac{f(q, D) \cdot (k_1 + 1)}{f(q, D) + k_1 \cdot (1 - b + b \cdot \frac{|D|}{\text{avgdl}})}$$

In [13]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

# Initialize the BM25 Retriever
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k = 2
print("type of bm25", type(bm25_retriever))


type of bm25 <class 'langchain_community.retrievers.bm25.BM25Retriever'>


In [14]:
# 여러개의 검색기를 결합하여, 다양한 검색 기법 활용. chroma 겁색기, bm25 검색기 결합. 각각 0.2, 0.8 가중치 적용
ensemble_retriever = EnsembleRetriever ( retrievers=[bm25_retriever, chroma_retriever], weights=[0.2, 0.8])


In [15]:
# 예시 고객 문의
query = "에코프로에 대해서 알려줘"

docs = ensemble_retriever.invoke(query)

docs

[Document(metadata={'producer': 'Skia/PDF m125', 'creator': 'Chromium', 'creationdate': '2024-07-03T03:24:25+00:00', 'source': 'sample.pdf', 'file_path': 'sample.pdf', 'total_pages': 4, 'format': 'PDF 1.4', 'title': '경제 : 네이버 뉴스', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-07-03T03:24:25+00:00', 'trapped': '', 'modDate': "D:20240703032425+00'00'", 'creationDate': "D:20240703032425+00'00'", 'page': 2}, page_content='에코프로, 포항 기회발전특구 지정\n에…"2조 추가 투자, 마더팩토리 구축"\n42분전\n\'로또\' 과천에 10만명 로또 아닌 마포도\n4만명 \'우르르\'\n43분전\n에이비엘바이오 "내년 말까지 최소 2\n개 기술이전…4000억 자금 확보"\n43분전\n‘2800선 회복’ 넘봤던 코스피, 장중 상\n승폭 반납… 반도체주 약세\n45분전\n물가 내렸지만 높아진 환율에…한은 금\n리인하 가능성은 \'글쎄\'\n46분전\n코스피, 2790선 회복 시도…S&P·나스\n닥 최고치 경신\n46분전\n하나은행 주춤하는 사이 국민·신한 달\n린다\n46분전\n기사 더보기\n뉴시스\n아시아경제\n조세일보\n이데일리\n문화일보\n문화일보\n문화일보\n데일리안\nSBS Biz\n아이뉴스24\n조선비즈\n한경비즈니스\nSBS Biz\n머니투데이\n조선비즈\n조세일보\nSBS Biz\n아이뉴스24\n로그인\n전체서비스\n서비스안내\n오류신고\n고객센터\n기사배열 책임자 : 김수향\n청소년 보호 책임자 : 이정규\n01:55\n02:09\n02:03'),
 Document(metadata={'producer':

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
다음 맥락을 바탕으로 질문에 답변하세요

{context}

질문: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Post-Processing
def format_docs(docs):
    formatted = "\n\n".join(doc.page_content for doc in docs)
    return formatted

# Chain
rag_chain = (
    {"context": ensemble_retriever| format_docs, "question": RunnablePassthrough()}
    | prompt
    |llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("에코프로에 대해서 알려줘")



'에코프로는 최근 포항 기회발전특구로 지정되었으며, 이와 관련하여 2조 원 규모의 추가 투자를 계획하고 있습니다. 이 투자에는 마더팩토리 구축이 포함되어 있어, 에코프로의 생산능력과 기술력을 강화하는 데 기여할 것으로 보입니다. 에코프로는 주로 환경 관련 기술 및 제품을 개발하는 기업으로, 지속 가능한 발전을 목표로 하고 있습니다.'